In [1]:
import geojson
import netCDF4 as nc
import os
import numpy as np
import xarray
import numpy.ma as ma

In [2]:
folder = r'p:\11205479-coclico\data\03_sea_level_jrc'
dataset2 = 'CoastAlRisk_Europe_ESL_RCP45.nc'
dataset3 = 'CoastAlRisk_Europe_ESL_RCP85.nc'
dataset = 'CoastAlRisk_Europe_ESL_Historical.nc'

In [3]:
ds = nc.Dataset(os.path.join(folder, dataset))
#ds
ds2 = nc.Dataset(os.path.join(folder, dataset2))
#ds2
ds2 = nc.Dataset(os.path.join(folder, dataset3))
#ds3

In [4]:
# reshaping the historical dataset to merge into the RCP45 and RCP85 one
dataset_new = 'CoastAlRisk_Europe_ESL_Historical_new.nc'
ds_new = nc.Dataset(os.path.join(folder, dataset_new), 'w')

# copy global attributes all at once via dictionary
ds_new.setncatts(ds.__dict__)

# copy dimensions
for name, dimension in ds.dimensions.items():
    ds_new.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))

# adding a new dimension
ds_new.createDimension('nsdec', (1)) # only for 1995

# copy all file data and extend for 2 instances
extended_var = ['esl', 'eewl']
for name, variable in ds.variables.items():
    if name in extended_var:
        ds_new.createVariable(name, variable.datatype, ('npoints','nrp','nens','nsdec'))
        ds_new[name].setncatts(ds[name].__dict__) # copy variable attributes all at once via dictionary
        ds_new[name][:] = np.expand_dims(ds[name][:], axis=3) # data, extended
    else:
        ds_new.createVariable(name, variable.datatype, variable.dimensions)
        ds_new[name].setncatts(ds[name].__dict__) # copy variable attributes all at once via dictionary
        ds_new[name][:] = ds[name][:] # data
        
# adding a new variables
ds_new.createVariable('decades', 'float32', ('nsdec',))
ds_new['decades'][:] = ma.masked_array(int(1995), mask=[0], dtype='float32')

C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarnin

In [5]:
# combining the datasets
ens_list_45 = [xarray.open_mfdataset(os.path.join(folder, dataset.split('.')[0]+'_new.nc')),
            xarray.open_mfdataset(os.path.join(folder, dataset2))]
ds_comb_45 = xarray.concat(ens_list_45, data_vars='different', dim='nsdec') # only concat files that are different

ens_list_85 = [xarray.open_mfdataset(os.path.join(folder, dataset.split('.')[0]+'_new.nc')),
            xarray.open_mfdataset(os.path.join(folder, dataset3))]
ds_comb_85 = xarray.concat(ens_list_85, data_vars='different', dim='nsdec') # only concat files that are different

In [6]:
# saving the new datasets
ds_comb_45.to_netcdf(os.path.join(folder, dataset2.split('.')[0]+'_new.nc')) # Export netcdf file
ds_comb_85.to_netcdf(os.path.join(folder, dataset3.split('.')[0]+'_new.nc')) # Export netcdf file

In [7]:
# loading new dataset
ds2 = nc.Dataset(os.path.join(folder, dataset2.split('.')[0]+'_new.nc'))
ds3 = nc.Dataset(os.path.join(folder, dataset3.split('.')[0]+'_new.nc'))

In [13]:
# get minima and maxima for the colormap boundaries
rp = ds2['rp'][:].filled()

for idx, i in enumerate(rp):
    print(i, min(ds3['esl'][:,idx,1].filled().flatten()), max(ds3['esl'][:,idx,1].filled().flatten()))

C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


5.0 0.4650539656031104 11.722749954526657
10.0 0.48766543080387137 11.85304010501605
20.0 0.5080505881690962 11.975193777702065
50.0 0.5322279771503171 12.129956660321566
100.0 0.548861282091625 12.243735736021819
200.0 0.5644194160947287 12.351218512870865
500.0 0.5838590944770807 12.484405731554357
1000.0 0.5981188276383687 12.578953862112536


In [5]:
# write data to file 
rp = ds['rp'][:].filled()

features = []
for j, (lon, lat) in enumerate(zip(ds['longitude'][:], ds['latitude'][:])):
    point = geojson.Point((float(lon), float(lat)))
    feature = geojson.Feature(geometry=point)
    feature['properties']['locationId'] = j
    feature['properties']['esl'] = np.array2string(ds['esl'][j][:,1].filled(), max_line_width=np.inf)
    feature['properties']['rp'] = str(rp)
    features.append(feature) 

# store the features
collection = geojson.FeatureCollection(features)
with open(os.path.join(folder, 'platform', r'EU_ESL_JRC_Hist_RP.geojson'), 'w') as f:
    geojson.dump(collection, f)

In [6]:
# check geojson content
with open(os.path.join(folder, 'platform', 'EU_ESL_JRC_Hist_RP.geojson')) as f:
    check = geojson.load(f)
    
check.keys()
check['features'][0]

{"geometry": {"coordinates": [-8.991017, 37.030349], "type": "Point"}, "properties": {"esl": "[3.30205426 3.41486462 3.52046817 3.64998335 3.74099187 3.82651434 3.93185498 4.00618858]", "locationId": 0, "rp": "[   5.   10.   20.   50.  100.  200.  500. 1000.]"}, "type": "Feature"}